In [1]:
import sys
sys.path.append('..')
import ast
import scipy.io
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import statistics
import pandas as pd
import pickle
import os
from constant import path, parameter
from lib import hypothesis

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from lib import evaluate
rules = evaluate.Rules()

In [4]:
dataframe = []
for i in range(len(path.CV_FILENAME)):
    result_file = os.path.join(path.RESULT_PATH,"result_{}.pickle".format(path.CV_FILENAME[i]))
    pickle_in = open(result_file,"rb")
    example_dict = pickle.load(pickle_in)
    dataframe.append(example_dict)
    
data = pd.DataFrame.from_dict(dataframe)
data

,Dataset,arrErrSum,arrErrProd,arrErrMin,arrErrMax
0,cv_australian.mat,"[0.14492753623188406, 0.2608695652173913, 0.24...","[0.14492753623188406, 0.2608695652173913, 0.24...","[0.15942028985507245, 0.2753623188405797, 0.28...","[0.15942028985507245, 0.2753623188405797, 0.28..."
1,cv_glass.mat,"[0.47619047619047616, 0.36363636363636365, 0.3...","[0.47619047619047616, 0.3181818181818182, 0.45...","[0.47619047619047616, 0.36363636363636365, 0.4...","[0.47619047619047616, 0.4090909090909091, 0.45..."


In [5]:
dict_data = []
for i in range(len(path.CV_FILENAME)):
    dict_data.append(hypothesis.mean_rules(path.CV_FILENAME[i],data['arrErrSum'][i],
                                      data['arrErrProd'][i],data['arrErrMin'][i],
                                      data['arrErrMax'][i]))

print('Bảng 1: Classification error của các fixed combining rule')
data_1 = pd.DataFrame.from_dict(dict_data)
data_1

Bảng 1: Classification error của các fixed combining rule


,Dataset,mean_sum,variance_sum,mean_product,variance_product,mean_min,variance_min,mean_max,variance_max
0,cv_australian.mat,0.206280,0.001140,0.210628,0.001068,0.213527,0.001998,0.212077,0.002049
1,cv_glass.mat,0.377417,0.008615,0.406926,0.005935,0.434848,0.005891,0.443218,0.015694


In [6]:
final_results = []
for i in range(len(path.CV_FILENAME)):
    # Sum vs Product
    win_compare_sum_prod = hypothesis.win_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    equal_compare_sum_prod = hypothesis.equal_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    loss_compare_sum_prod = hypothesis.loss_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    # Sum vs Min
    win_compare_sum_min = hypothesis.win_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    equal_compare_sum_min = hypothesis.equal_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    loss_compare_sum_min = hypothesis.loss_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    # Sum vs Max
    win_compare_sum_max = hypothesis.win_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    equal_compare_sum_max = hypothesis.equal_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    loss_compare_sum_max = hypothesis.loss_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    dict_result = {'Dataset':path.CV_FILENAME[i],'win_sum_prod':win_compare_sum_prod,'equal_sum_prod':equal_compare_sum_prod,
                  'loss_sum_prod':loss_compare_sum_prod,'win_sum_min':win_compare_sum_min,'equal_sum_min':equal_compare_sum_min,
                  'loss_sum_min':loss_compare_sum_min,'win_sum_max':win_compare_sum_max,'equal_sum_max':equal_compare_sum_max,
                  'loss_sum_max':loss_compare_sum_max}
    final_results.append(dict_result)
    
print('Bảng 2: So sanh Win - Equal - Loss')
data_2 = pd.DataFrame.from_dict(final_results)
data_2

Bảng 2: So sanh Win - Equal - Loss


,Dataset,win_sum_prod,equal_sum_prod,loss_sum_prod,win_sum_min,equal_sum_min,loss_sum_min,win_sum_max,equal_sum_max,loss_sum_max
0,cv_australian.mat,6,24,0,13,10,7,13,10,7
1,cv_glass.mat,13,8,9,16,14,0,22,5,3


In [7]:
for cv in path.CV_FILENAME:
    hypothesis.wilcoxon(cv_file=cv, data=data, number_cv = len(path.CV_FILENAME))    

------------ Kiem dinh Wilcoxon ------------
Data:  cv_australian.mat
WilcoxonResult(statistic=0.0, pvalue=0.02435791571803699)
WilcoxonResult(statistic=52.5, pvalue=0.015711813204806174)
------------ Kiem dinh Wilcoxon ------------
Data:  cv_glass.mat
WilcoxonResult(statistic=0.0, pvalue=0.02435791571803699)
WilcoxonResult(statistic=52.5, pvalue=0.015711813204806174)
